In [ ]:
import pyodbc
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import json

In [ ]:
r = requests.get("https://github.com/owid/covid-19-data/tree/master/public/data",allow_redirects=True)
r = r.content

In [ ]:
# create soup object
soup = bs(r,"html.parser")

In [ ]:
# export html doc to file stored
with open('response.html','wb') as file:
    file.write(soup.prettify('utf-8'))

In [ ]:
# search & navigate html tree to extract data
links = soup.find_all('a',class_="js-navigation-open Link--primary",title='owid-covid-data.csv')

In [ ]:
#extract text
link = soup.find('a',class_="js-navigation-open Link--primary",title='owid-covid-data.csv')
link.text

In [ ]:
base_url = 'https://raw.githubusercontent.com'

In [ ]:
# extract data from html tree to get attribute value of href file link
for i in links:
    r_url = repr(i.get('href'))
    url_1 = base_url.strip("'")+r_url.strip("'")
    url = url_1.strip('/blob')
    print(url)

In [ ]:
response= requests.get("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv")
response.status_code

In [ ]:
open(url.rsplit('/', 1)[1], 'wb').write(response.content)

In [ ]:
url.rsplit('/', 1)[1]

In [ ]:
# r = json.loads(response.content)

In [ ]:
# covid_data=pd.DataFrame.from_dict(r,orient='index')
# covid_data.head()

In [ ]:
covid_data = pd.read_csv('owid-covid-data.csv')

In [ ]:
covid_data.columns

In [ ]:
covid_death = covid_data[['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million']]
covid_death = covid_death.fillna(value=0)
covid_death.head()

In [ ]:
covid_vaccination = covid_data[['iso_code', 'continent', 'location', 'date','total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations', 'new_vaccinations_smoothed',
       'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred',
       'people_fully_vaccinated_per_hundred', 'total_boosters_per_hundred',
       'new_vaccinations_smoothed_per_million',
       'new_people_vaccinated_smoothed',
       'new_people_vaccinated_smoothed_per_hundred']]
covid_vaccination = covid_vaccination.fillna(value=0)
covid_vaccination.tail().sort_values(by='date', ascending=False)

In [ ]:
connection = pyodbc.connect('Driver={SQL Server};'
                      'Server=LAPTOP-E28526DD\SQLEXPRESS;'
                      'Database=coviddata;'
                      'Trusted_Connection=yes;')

In [ ]:
cursor = connection.cursor()

In [ ]:
# cursor.execute("DROP TABLE IF EXISTS [covid_death$];")
# cursor.execute("DROP TABLE IF EXISTS [covid_vaccination$];")

In [ ]:
# cursor.execute("""
#   CREATE TABLE covid_death (
#     id serial PRIMARY KEY,
#     description VARCHAR NOT NULL
#   );
# """)

In [ ]:
# initial data load
for index, row in covid_death.iterrows():
    cursor.execute("INSERT INTO [coviddata].[dbo].[covid_death$]\
                   (iso_code, continent, location, date, total_cases, new_cases,new_cases_smoothed, total_deaths, new_deaths\
                   ,new_deaths_smoothed, total_cases_per_million,new_cases_per_million, new_cases_smoothed_per_million\
                   ,total_deaths_per_million, new_deaths_per_million,new_deaths_smoothed_per_million) \
                   values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)" \
                   ,row.iso_code, row.continent, row.location, row.date, row.total_cases \
                   ,row.new_cases,row.new_cases_smoothed, row.total_deaths \
                   ,row.new_deaths,row.new_deaths_smoothed, row.total_cases_per_million \
                   ,row.new_cases_per_million, row.new_cases_smoothed_per_million,row.total_deaths_per_million \
                   ,row.new_deaths_per_million,row.new_deaths_smoothed_per_million)
connection.commit()

In [ ]:
# initial data load
for index, row in covid_vaccination.iterrows():
    cursor.execute("INSERT INTO [coviddata].[dbo].[covid_vaccination$] \
    (iso_code, continent, location, date,total_vaccinations,people_vaccinated, people_fully_vaccinated \
                   , total_boosters,new_vaccinations, new_vaccinations_smoothed,total_vaccinations_per_hundred \
                   , people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred, total_boosters_per_hundred \
                   , new_vaccinations_smoothed_per_million,new_people_vaccinated_smoothed \
                   , new_people_vaccinated_smoothed_per_hundred) \
                   values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)" \
                   , row.iso_code, row.continent, row.location, row.date  \
                   , row.total_vaccinations,row.people_vaccinated, row.people_fully_vaccinated  \
                   , row.total_boosters,row.new_vaccinations, row.new_vaccinations_smoothed \
                   ,row.total_vaccinations_per_hundred, row.people_vaccinated_per_hundred \
                   ,row.people_fully_vaccinated_per_hundred, row.total_boosters_per_hundred \
                   , row.new_vaccinations_smoothed_per_million,row.new_people_vaccinated_smoothed \
                   , row.new_people_vaccinated_smoothed_per_hundred)
connection.commit()

In [ ]:
## truncate & create temp table

cursor.execute("truncate table [coviddata].[dbo].[covid_death_t$]")

for index, row in covid_death.iterrows():
    cursor.execute("
               select iso_code, continent, location, date, total_cases, new_cases,new_cases_smoothed, total_deaths, new_deaths \
                   ,new_deaths_smoothed, total_cases_per_million,new_cases_per_million, new_cases_smoothed_per_million \
                   ,total_deaths_per_million, new_deaths_per_million,new_deaths_smoothed_per_million \
                into #[coviddata].[dbo].[covid_death_t$] 
                from covid_death
               ")
connection.commit()

In [ ]:
## truncate & create temp table

cursor.execute("truncate table [coviddata].[dbo].[covid_vaccination_t$]")

for index, row in covid_vaccination.iterrows():
    cursor.execute("
               select iso_code, continent, location, date,total_vaccinations,people_vaccinated
               , people_fully_vaccinated, total_boosters,new_vaccinations, new_vaccinations_smoothed
               ,total_vaccinations_per_hundred, people_vaccinated_per_hundred
               ,people_fully_vaccinated_per_hundred, total_boosters_per_hundred
               , new_vaccinations_smoothed_per_million,new_people_vaccinated_smoothed
               , new_people_vaccinated_smoothed_per_hundred
    into #[coviddata].[dbo].[covid_vaccination_t$]
    from covid_vaccination
               ")
connection.commit()

In [ ]:
## for coming iterations of updating change, insert difference
cursor.execute(
"insert into [coviddata].[dbo].[covid_death$](iso_code, continent, location, date \
    , total_cases, new_cases,new_cases_smoothed, total_deaths, new_deaths \
    ,new_deaths_smoothed, total_cases_per_million,new_cases_per_million \
    , new_cases_smoothed_per_million,total_deaths_per_million, new_deaths_per_million,new_deaths_smoothed_per_million) \
(select * from [coviddata].[dbo].[covid_death$] as a \
except \
select * from [coviddata].[dbo].[covid_death_t$] as b) \
)"
)

cursor.execute(
"insert into [coviddata].[dbo].[covid_vaccination$](iso_code, continent \
, location, date, total_cases, new_cases,new_cases_smoothed \
, total_deaths, new_deaths,new_deaths_smoothed, total_cases_per_million  \
,new_cases_per_million, new_cases_smoothed_per_million,total_deaths_per_million  \
, new_deaths_per_million,new_deaths_smoothed_per_million) \
(select * from [coviddata].[dbo].[covid_vaccination$] as a  \
except \
select * from [coviddata].[dbo].[covid_vaccination_t$] as b) \
)"
)

In [ ]:
cursor.execute('SELECT top 5 * FROM [coviddata].[dbo].[covid_vaccination$]')

In [ ]:
result = cursor.fetchall() # retrieve previous statement
print(result)

In [ ]:
connection.close()
cursor.close()